# Synthesis Planning
This notebook goes through the whole process to use the optimized model and produce the synthesis planning.

## Environment Setup
First of all, we need to make sure that the notebook is running in the correct environment.

To do that, follow these steps:
 1. Create the project's environment
    To do that, place yourself in the project's root, and run :
    `conda env create -f environment.yml`
    This creates a new clean conda environment with the package needed by the project.
 2. Activate the environment
    On Linux and Mac:
    `source activate synnet`
    On Windows:
    `conda activate synnet`
 3. Install the project's module
    Now that the environment is activated, we need to install the project as a module.
    Place yourself in the project's root and run :
    `pip install -e .`
 4. Restart Jupyter from the new environment
    Now, we can start Jupyter from the environment, that way it has all the dependencies we need. Simply run `jupyter notebook` and open this notebook.

To test the setup, run the following cell.

In [ ]:
import sys

# Check that the correct conda env is being used
if sys.prefix.split("\\")[-1] != "synnet":
    print("You are not using the correct conda environment, please follow the instructions above")
else:
    try:
        import synnet

        print("The environment is setup correctly")
    except ImportError:
        print("The module 'synnet' is not installed, please follow the instructions above")

## Pre-Processing

Now that the conda environment is correctly setup, we can start the preliminary steps to produce the synthesis results.

First, let's import some packages, define some constants.
Make sure they are correct.

In [ ]:
from downloader import *
from preprocessor import *
from synthesis import synthesis
import random

project_root = Path("..")  # Path to the project's root folder
cpu_cores = 6  # Number of cores to use for the computation. The greater, the faster
num_samples = 50  # Number of molecules to randomly pick from the datasets

random.seed(2022)

### Load data
First, we need to choose the trained model to use.

For now, we only have the one provided by the paper's authors

In [ ]:
original_checkpoints = get_original_checkpoints(project_root)

Now, we need to retrieve the building blocks. We asked the company to provide them, that way we can correctly reproduce their result

To simplify the workflow, this also perform the step 0 described in INSTRUCTIONS.md

In [ ]:
bblocks_raw = get_building_blocks(project_root)

We also need to download the molecules we want to test the model on.

We will use two datasets : ZINC and ChEMBL

In [ ]:
zinc_smiles = get_zinc_dataset(project_root, num_samples)
chembl_smiles = get_chembl_dataset(project_root, num_samples)

## Process Building Blocks

### Filter Building Blocks
First, we apply the step 1 from INSTRUCTION.md

We pre-process the building blocks to identify applicable reactants for each reaction template. In other words, filter out all building blocks that do not match any reaction template. There is no need to keep them, as they cannot act as reactant.

In a first step, we match all building blocks with each reaction template.
In a second step, we save all matched building blocks and a collection of `Reactions` with their available building blocks.

In [ ]:
bblocks, rxn_collection = filter_bblocks(project_root, bblocks_raw)

### Pre-compute embeddings
Then, step 2

We use the embedding space for the building blocks a lot. Hence, we pre-compute and store the building blocks.

In [ ]:
mol_embedder = compute_embeddings(project_root, bblocks, cpu_cores)

# Synthesis

Now that everything is loaded and pre-processed, we can to the synthesis prediction.

First, we compute synthetic trees for a random sample of 10000 molecules from the original building blocks list.

In [ ]:
sample_bblocks = random.sample(bblocks, 10000)

synthesis(sample_bblocks,
          bblocks,
          original_checkpoints,
          rxn_collection,
          mol_embedder,
          project_root / "results" / "bblocks" / "synthesis" / "original",
          rxn_template="hb",
          n_bits=4096,
          beam_width=3,
          max_step=15,
          cpu_cores=cpu_cores)

Compute synthetic trees for the ZINC dataset

In [ ]:
synthesis(zinc_smiles,
          bblocks,
          original_checkpoints,
          rxn_collection,
          mol_embedder,
          project_root / "results" / "zinc" / "synthesis" / "original",
          rxn_template="hb",
          n_bits=4096,
          beam_width=3,
          max_step=15,
          cpu_cores=cpu_cores)

Compute synthetic trees for the ChEMBL dataset

In [ ]:
synthesis(chembl_smiles,
          bblocks,
          original_checkpoints,
          rxn_collection,
          mol_embedder,
          project_root / "results" / "chembl" / "synthesis" / "original",
          rxn_template="hb",
          n_bits=4096,
          beam_width=3,
          max_step=15,
          cpu_cores=cpu_cores)